<a href="https://colab.research.google.com/github/CSF0744/ECE171B/blob/main/Segway.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install control
!pip install slycot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 398 kB 19.4 MB/s 
  Created wheel for control: filename=control-0.9.2-py2.py3-none-any.whl size=403205 sha256=48751bdcf2389a7511e8b0263736d6a6a5b59a60361e8a71bbd7330d2eddcfda
  Stored in directory: /root/.cache/pip/wheels/48/ef/c2/929bb5c59a1328df00a0561d0e68cd7c8537f33f5ce0ce741b
Successfully built control
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.0 MB 23.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for slycot: filename=slycot-0.5.0-cp37-cp37m-linux_x86_64.whl size=2021298 sha256=b32bb25a63df14d82c7e019ed2abbb0c5a964f5332aef5d3b7ef44c6acbccd13
  Stored in directory: /root/.cache/pip/wheels/7f/2e/54/e2cb7cf358f956ed0efa9923ab4f5aa353a941422c55b25531

In [ ]:
import numpy as np
from google.colab.patches import cv2_imshow
import cv2
from scipy.integrate import solve_ivp
import control
import code
from PIL import Image
from IPython.display import display, clear_output
from numpy.linalg import matrix_rank
from numpy import linalg as LA
import math

In [ ]:
# Visualize cart and pendulum movement via OpenCV 
# Source Code: https://github.com/mpkuse/inverted_pendulum/blob/master/InvertedPendulum.py
class CartPendulum:
    def __init__(self):
        f = 0

    def step( self, state_vec, t=None ):
        """ state vector :
                x0 : position of the cart
                x1 : veclocity of the cart
                x2 : angle of pendulum. In ref frame with x as forward of the cart and y as up. Angile with respect to ground plane
                x3 : angular velocity of the pendulum
        """
        CART_POS = state_vec[0]
        BOB_ANG  = state_vec[1]*180. / np.pi # degrees
        
        LENGTH_OF_PENDULUM = 110.

        IM = np.zeros( (512, 512,3), dtype='uint8' )

        # Ground line
        cv2.line(IM, (0, 450), (IM.shape[1], 450), (19,69,139), 4 )


        # Mark ground line
        XSTART = -5.
        XEND = 5.
        for xd in np.linspace( XSTART, XEND, 11 ):
            x = int(   (xd - XSTART) / (XEND - XSTART) * IM.shape[0]   )

            cv2.circle( IM, (x, 450), 5, (0,255,0), -1 )

            cv2.putText(IM, str(xd), (x-15,450+15), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,250), 1);


        # Draw Wheels of the cart
        wheel_1_pos = int(   (CART_POS - 1.2 - XSTART) / (XEND - XSTART) * IM.shape[0]   )
        wheel_2_pos = int(   (CART_POS + 1.2 - XSTART) / (XEND - XSTART) * IM.shape[0]   )

        cv2.circle( IM, (wheel_1_pos, 415), 25, (255,255,255), 6 )
        cv2.circle( IM, (wheel_2_pos, 415), 25, (255,255,255), 6 )
        cv2.circle( IM, (wheel_1_pos, 415), 2, (255,255,255), -1 )
        cv2.circle( IM, (wheel_2_pos, 415), 2, (255,255,255), -1 )

        # Cart base
        cart_base_start = int(   (CART_POS - 2.5 - XSTART) / (XEND - XSTART) * IM.shape[0]   )
        cart_base_end   = int(   (CART_POS + 2.5 - XSTART) / (XEND - XSTART) * IM.shape[0]   )

        cv2.line( IM, (cart_base_start, 380), (cart_base_end, 380), (255,255,255), 6 )

        # Pendulum hinge
        pendulum_hinge_x = int(   (CART_POS - XSTART) / (XEND - XSTART) * IM.shape[0]   )
        pendulum_hinge_y = 380
        cv2.circle( IM, (pendulum_hinge_x, pendulum_hinge_y), 10, (255,255,255), -1 )


        # Pendulum
        pendulum_bob_x = int( LENGTH_OF_PENDULUM * np.cos( (BOB_ANG+90) / 180. * np.pi ) )
        pendulum_bob_y = int( LENGTH_OF_PENDULUM * np.sin( (BOB_ANG+90) / 180. * np.pi ) )
        cv2.circle( IM, (pendulum_hinge_x+pendulum_bob_x, pendulum_hinge_y-pendulum_bob_y), 10, (255,255,255), -1 )
        cv2.line( IM, (pendulum_hinge_x, pendulum_hinge_y), (pendulum_hinge_x+pendulum_bob_x, pendulum_hinge_y-pendulum_bob_y), (255,255,255), 3 )

        # Mark the current angle
        angle_display = BOB_ANG % 360
        if( angle_display > 180 ):
            angle_display = -360+angle_display
        cv2.putText(IM, "theta="+str( np.round(angle_display,4) )+" deg", (pendulum_hinge_x-15, pendulum_hinge_y-15), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,250), 1);


        # Display on top
        if t is not None:
            cv2.putText(IM, "t="+str(np.round(t,4))+"sec", (15, 15), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,250), 1);
            cv2.putText(IM, "ANG="+str(np.round(BOB_ANG,4))+" degrees", (15, 35), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,250), 1);
            cv2.putText(IM, "POS="+str(np.round(CART_POS,4))+" m", (15, 55), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,250), 1);



        return IM

### Define the nonlinear cart-pendulum dynamics model

In [ ]:
def x_dot(t, x):
    g = 9.8 #gravitational acceleration
    L = 1.0 #rider's leg length

    m = 30.0 #rider's mass 
    M = 10.0  #mass of cart 
    
    J = 100 #rider's moment-of-inertia 
    M_t = M+m
    J_t = J+m*L**2

    # When you need the control input in the dynamics, write it as u(t,x) so 
    # you can test the performance of different controllers 

    p_dot = x[2]
    
    theta_dot = x[3]
    
    p_ddot = (J_t*u(t,x)-J_t*m*L*math.sin(x[1])*x[3]*x[3]+m*m*g*L*L*math.sin(x[1])*math.cos(x[1]))/(M_t*J_t-m*m*L*L*math.cos(x[1])*math.cos(x[1]))

    theta_ddot = (m*L*math.cos(x[1])*u(t,x)-m*m*L*L*math.sin(x[1])*math.cos(x[1])*x[3]*x[3]+M_t*m*g*L*math.sin(x[1]))/(M_t*J_t-m*m*L*L*math.cos(x[1])*math.cos(x[1]))

    return [p_dot, theta_dot, p_ddot, theta_ddot]

### Simulate the system without control

In [ ]:
# No control
def u(t, x):
    return 0

In [ ]:
# Simulate the dynamical system
from google.colab import files
if __name__=="__main__":
    sol = solve_ivp(x_dot, [0, 20], [1.0, 0+0.05, 0., 0.],  t_eval=np.linspace(0, 20, 100))

    syst = CartPendulum()
    out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (512,512))
    for i, t in enumerate(sol.t):
        rendered = syst.step([sol.y[0,i], sol.y[1,i], sol.y[2,i], sol.y[3,i]], t)
        out.write(rendered);

        if cv2.waitKey(25) == ord('q'):
            break
    out.release()
    files.download('outpy.avi')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Controller Design

It can be observed from the previous simulation, that if there is no control, the pendulum is going to swing and swing, and will not stop since we assume the friction coefficient is 0. This is the system we want to control.

Now, we are going to design controllers based on the system linearization and then apply them to the full nonlinear dynamical system.

The first thing we need to do is to get the linearization Jacobian $\frac{\partial f}{\partial x}$ and $\frac{\partial f}{\partial u}$. Then we plug in the equilibrium point we want to linearize the system around and get the A, B matrix.

Before we design a stablizing controller, the first thing we want to do is to check whether then system is reachable/controllable.

### Linearization

In [ ]:
class LinearizedSystem:
    def __init__(self):
        g = 9.8 #gravitational acceleration
        L = 1.0 #rider's leg length

        m = 30.0 #rider's mass 
        M = 10.0  #mass of cart 
        
        J = 100 #rider's moment-of-inertia 

        # Pendulum up (linearized eq)
        M_t = M+m
        J_t = J+m*L**2
        mu = M_t*J_t - m**2*L**2

        # Linearized system around theta = 0
        self.A = np.array([[0, 0, 1., 0],\
                           [0, 0, 0, 1.],\
                           [0, m**2*L**2*g/mu, 0, 0],\
                           [0, M_t*m*g*L/mu, 0, 0]])

        self.B = np.expand_dims( np.array( [0, 0, J_t/mu, m*L/mu]) , 1 ) # 4x1

### Compute the reachability matrix W_r

In [ ]:
ss = LinearizedSystem()

eig, _ = LA.eig(ss.A)

# You will notice the original system is unstable since the eigen(A) has positive real part
print('Eigenvalue of A matrix', eig)

# Let's compute the reachability matrix W_r
W_r = 

# Check the rank of W_r, is the system reachable?
print('Rank of reachability matrix', matrix_rank(W_r))

### Eigenvalue placement 

In [ ]:
## If the system is reachable, now we can design state feedback controller to stablize the system 
desired_eigs = np.array([-1.1, -1.2, -1.3, -1.4])

# Use eigenvalue placement to compute compute K such that the closed-loop system
# has the desired eigenvalues eig([A-BK]) = desired_eigs
K_stable = None

print('[compute_K] desired_eigs=', desired_eigs)
print('[compute_K] K value=', K_stable)

In [ ]:
# Now we will test the control law u = -Kx, where K is solved from the eigenvalue placement
def u(t, x):
    u_ = -np.matmul(K_stable, x-np.array([0, 0, 0, 0]))
    return u_[0]

In [ ]:
# Simulate the dynamical system
from google.colab import files
if __name__=="__main__":
    sol = solve_ivp(x_dot, [0, 20], [1.0, 0+0.05, 0., 0.],  t_eval=np.linspace(0, 20, 100))

    syst = CartPendulum()
    out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (512,512))
    for i, t in enumerate(sol.t):
        rendered = syst.step([sol.y[0,i], sol.y[1,i], sol.y[2,i], sol.y[3,i]], t)
        out.write(rendered);

        if cv2.waitKey(25) == ord('q'):
            break
    out.release()
    files.download('outpy.avi')

### LQR Controller

Where should we optimally place the eigenvalues?

In [ ]:
# State and Control Cost Matrix
Q = np.diag([1., 1., 1., 1.])
R = np.diag([0.001])

# K_lqr : State feedback gain
# P : Solution to Riccati Equation
# E_lqr : Eigen values of the closed loop system

# define how to compute the lqr gain K
K_lqr, P, E_lqr = None

print('[LQR] K value=', K_lqr)
print('[LQR] eigenvalues of the closed-loop system =', E_lqr)

In [ ]:
# Now we will test the LQR Controller.
def u(t, x):
    u_ = -np.matmul(K_lqr, x - np.array([0, 0, 0, 0])) 
    return u_[0]

In [ ]:
# Simulate the dynamical system
from google.colab import files
if __name__=="__main__":
    sol = solve_ivp(x_dot, [0, 20], [1.0, 0+0.05, 0., 0.],  t_eval=np.linspace(0, 20, 100))

    syst = CartPendulum()
    out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (512,512))
    for i, t in enumerate(sol.t):
        rendered = syst.step([sol.y[0,i], sol.y[1,i], sol.y[2,i], sol.y[3,i]], t)
        out.write(rendered);

        if cv2.waitKey(25) == ord('q'):
            break
    out.release()
    files.download('outpy.avi')